In [1]:
%load_ext autoreload
%autoreload 2
import manenv
from manenv.utils import *
from manenv.world import World
from manenv.component import *
from manenv.render_wrapper import RenderWrapper
from manenv.product import *

pygame 2.5.2 (SDL 2.28.3, Python 3.12.2)
Hello from the pygame community. https://www.pygame.org/contribute.html


In [2]:
world = World((6, 6))
world.place_component(make_vector(1, 1), Spawner(world))

s = np.zeros((5, 5), dtype=np.int8)
s[1][2] = 1
s[2][3] = 3
product = Product(s)
world.place_product(make_vector(2, 2), product)


wrapper = RenderWrapper(world)
wrapper.render()